In [1]:
# Import all necessary libraries.
import os
import sys
import matplotlib.image as mpimg
import warnings
warnings.filterwarnings('ignore')


# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')

# specify the path to Mask_RCNN if you placed it outside Nomeroff-net project
MASK_RCNN_DIR = os.path.join(NOMEROFF_NET_DIR, 'Mask_RCNN')

MASK_RCNN_LOG_DIR = "../logs/"
MASK_RCNN_MODEL_PATH = "../models/mask_rcnn_numberplate_0700.h5"
OPTIONS_MODEL_PATH =  "../models/numberplate_options_2019_2_15.h5"
OCR_NP_UKR_TEXT =  "../models/anpr_ocr_ua_1_2_11-cpu.h5"
OCR_NP_EU_TEXT =  "../models/anpr_ocr_eu_2-cpu.h5"

sys.path.append(NOMEROFF_NET_DIR)

# Import license plate recognition tools.
from NomeroffNet import  filters, RectDetector, TextDetector,  OptionsDetector, Detector, textPostprocessing
from NomeroffNet.TextDetectors.ocr_eu import ocr_eu

Using TensorFlow backend.


In [2]:
import numpy as np
import cv2
import asyncio
import time

In [3]:
class my_ocr_eu(ocr_eu):
    def __init__(self):
        ocr_eu.__init__(self)
        
    def normalize(self, img):
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (self.IMG_W, self.IMG_H))
        img = img.astype(np.float32)
        img -= np.amin(img)
        img /= np.amax(img)
        img = [[[h] for h in w] for w in img.T]

        x = np.zeros((self.IMG_W, self.IMG_H, 1))
        x[:, :, :] = img
        return x
    
    def predict(self, imgs):
        Xs = []
        for img in imgs:
            x = self.normalize(img)
            Xs.append(x)
        net_out_value = self.MODEL.predict(np.array(Xs))
        pred_texts = self.decode_batch(net_out_value)
        return pred_texts

In [4]:
class MyOptionsDetector(OptionsDetector):
    def __init__(self):
        OptionsDetector.__init__(self)
        
    def predict(self, imgs):
        Xs = []
        for img in imgs:
            Xs.append(self.normalize(img))
        predicted = self.MODEL.predict(np.array(Xs))

        regionIds = []
        for region in predicted[0]:
            regionIds.append(int(np.argmax(region)))
        
        stateIds = []
        for state in predicted[1]:
            stateIds.append(int(np.argmax(state)))
            
        return regionIds, stateIds
    
    def getRegionLabels(self, indexes):
        return [self.CLASS_REGION[index] for index in indexes]

In [5]:
nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)
nnet.loadModel(MASK_RCNN_MODEL_PATH)
rectDetector = RectDetector()

euTextDetector = my_ocr_eu()
euTextDetector.load(OCR_NP_EU_TEXT)
euTextDetector2 = ocr_eu()
euTextDetector2.load(OCR_NP_EU_TEXT)

optionsDetector = MyOptionsDetector()
optionsDetector.load(OPTIONS_MODEL_PATH)
optionsDetector2 = OptionsDetector()
optionsDetector2.load(OPTIONS_MODEL_PATH)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [6]:
# Detect numberplate
img_path = '../examples/images/example2.jpg'
img = mpimg.imread(img_path)
NP = nnet.detect([img])

In [7]:
async def test_1(): # put to model 2 np
    cv_img_masks = filters.cv_img_mask(NP)
    zones = []
    for img_mask in cv_img_masks:
        points = rectDetector.detect(img_mask)
        zone = rectDetector.get_cv_zonesBGR(img, points)
        zones.append(zone)

    regionIds, stateIds = optionsDetector.predict(zones)
    regionNames = optionsDetector.getRegionLabels(regionIds)
    return euTextDetector.predict(zones)


async def test_2(): # stay how to be
    cv_img_masks = filters.cv_img_mask(NP)
    texts = []
    for img_mask in cv_img_masks:
        points = rectDetector.detect(img_mask)
        zone = rectDetector.get_cv_zonesBGR(img, points)

        regionId, stateId = optionsDetector2.predict(zone)
        regionName = optionsDetector2.getRegionLabel(regionId)

        text = euTextDetector2.predict(zone)
        texts.append(text)
    return texts


async def async_func(img_mask): # async 
    points = rectDetector.detect(img_mask)
    zone = rectDetector.get_cv_zonesBGR(img, points)
    regionId, stateId = optionsDetector2.predict(zone)
    regionName = optionsDetector2.getRegionLabel(regionId)
    return euTextDetector2.predict(zone)       
async def test_3(): # async test
    texts = []
    loop = asyncio.get_event_loop()
    promises = []
    cv_img_masks = filters.cv_img_mask(NP)
    for img_mask in cv_img_masks:
        promise = loop.create_task(async_func(img_mask))
        promises.append(promise)
    if bool(promises):
        await asyncio.wait(promises)
    for promise in promises:
        texts.append(promise.result())
    return texts

In [8]:
COUNT_ITERATIONS = 16

In [9]:
start_time = time.time()
for i in range(COUNT_ITERATIONS):
    await test_1()    
print(f'TEST (2 input): {time.time() - start_time}')

TEST (2 input): 14.026314735412598


In [10]:
start_time = time.time()
for i in range(COUNT_ITERATIONS):
    await test_2()    
print(f'TEST (how to be): {time.time() - start_time}')

TEST (how to be): 14.354429721832275


In [11]:
start_time = time.time()
for i in range(COUNT_ITERATIONS):
    await test_3()    
print(f'TEST (async): {time.time() - start_time}')

TEST (async): 13.410621404647827


In [12]:
# async

In [13]:
start_time = time.time()
loop = asyncio.get_event_loop()
promises = []
for i in range(COUNT_ITERATIONS):
    promise = loop.create_task(test_1())
    promises.append(promise)
    
if bool(promises):
    await asyncio.wait(promises)
print(f'TEST (2 input): {time.time() - start_time}')

TEST (2 input): 13.145161628723145


In [14]:
start_time = time.time()
loop = asyncio.get_event_loop()
promises = []
for i in range(COUNT_ITERATIONS):
    promise = loop.create_task(test_2())
    promises.append(promise)
    
if bool(promises):
    await asyncio.wait(promises)
print(f'TEST (how to be): {time.time() - start_time}')

TEST (how to be): 12.776089668273926


In [15]:
start_time = time.time()
loop = asyncio.get_event_loop()
promises = []
for i in range(COUNT_ITERATIONS):
    promise = loop.create_task(test_3())
    promises.append(promise)
    
if bool(promises):
    await asyncio.wait(promises)
print(f'TEST (async): {time.time() - start_time}')

TEST (async): 12.657196283340454
